In [20]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
def tileset_info():
    return {
        'min_pos': [-2.5, -2.5],
        'max_pos': [2.5, 2.5],
        'bins_per_dimension': 256,
        'max_width': 5,
        'max_zoom': 50,
        'mirror_tiles': 'false',
    }

In [22]:
from numba import jit, vectorize, guvectorize, float64, complex64, int32, float32
import numpy as np

# from: https://www.ibm.com/developerworks/community/blogs/jfp/entry/How_To_Compute_Mandelbrodt_Set_Quickly?lang=en

@jit(int32(complex64, int32))
def mandelbrot(c,maxiter):
    nreal = 0
    real = 0
    imag = 0
    for n in range(maxiter):
        nreal = real*real - imag*imag + c.real
        imag = 2* real*imag + c.imag
        real = nreal;
        if real * real + imag * imag > 4.0:
            return n
    return 0

@guvectorize([(complex64[:], int32[:], int32[:])], '(n),()->(n)')
def mandelbrot_numpy(c, maxit, output):
    #print("hellow")
    maxiter = maxit[0]
    for i in range(c.shape[0]):
        output[i] = mandelbrot(c[i],maxiter)

"""
import numexpr as ne

def mandelbrot_numpy(c, maxiter):
    output = np.zeros(c.shape)
    z = np.zeros(c.shape, np.complex64)
    for it in range(maxiter):
        notdone = ne.evaluate('z.real*z.real + z.imag*z.imag < 4.0')
        output[notdone] = it
        z = ne.evaluate('where(notdone,z**2+c,z)')
    output[output == maxiter-1] = 0    
    return output
  """      
def mandelbrot_set2(xmin,xmax,ymin,ymax,width=256,height=256,maxiter=500):
    #print("hi")
    r1 = np.linspace(xmin, xmax, width, dtype=np.float32)
    r2 = np.linspace(ymin, ymax, height, dtype=np.float32)
    c = r1 + r2[:,None]*1j
    n3 = mandelbrot_numpy(c,maxiter)
    return n3

#%timeit mandelbrot_set2(-0.74877,-0.74872,0.06505,0.06510,256,256,100)
%time mandelbrot_set2(-0.74877,-0.74872,0.06505,0.06510,256,256,100)
%time mandelbrot_set2(-0.74877,-0.74872,0.06505,0.06510,1024,1024,100)
#%time mandelbrot_set2(-2.5, 2.5, -2.5, 2.5)

CPU times: user 31.7 ms, sys: 6.45 ms, total: 38.1 ms
Wall time: 71.3 ms
CPU times: user 505 ms, sys: 20.2 ms, total: 525 ms
Wall time: 741 ms


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [23]:
%%time 

import functools as ft
import numpy as np
import hgflask as hgf
import hgflask.client as hgc
import hgtiles.format as hgfo
import hgtiles.utils as hgut
import os.path as op

# from https://plus.maths.org/content/computing-mandelbrot-set

def tiles(tsinfo, z, x, y):
    [from_x, from_y, to_x, to_y] = hgut.tile_bounds(tsinfo, z, x, y)
    tile_size = tsinfo['bins_per_dimension'] if 'bins_per_dimension' in tsinfo else 256
    limit = 4
    
    #print("from_x", from_x, to_x, from_y, to_y)
    #points = mandelbrot(from_x, from_y, to_x, to_y, tile_size)
    points = mandelbrot_set2(from_x, to_x, from_y, to_y)
    #print("points:", points)

    
    # ravel because the points are returned as a 2D array
    #return points.ravel()
    return hgfo.format_dense_tile(points.ravel())
    
tilesets = [{
    'uuid': 'a',
    'handlers': {
        'tileset_info': tileset_info,
        'tiles': ft.partial(hgut.tiles_wrapper_2d, 
                            tiles_function=ft.partial(tiles, tileset_info()))
    }},
    {
        'uuid': 'b',
        'filepath': op.expanduser('~/Dropbox/paper-data/gaia/cooler.mcool')
    }
]

server=hgf.start(tilesets)
#points = tiles(tsinfo, 1, 1, 1)

terminating: X99lsgMCTjWBQqxtM0J_4Q


 * Running on http://0.0.0.0:55432/ (Press CTRL+C to quit)


sleeping
info: {'a': {'min_pos': [-2.5, -2.5], 'max_pos': [2.5, 2.5], 'bins_per_dimension': 256, 'max_width': 5, 'max_zoom': 50, 'mirror_tiles': 'false'}}


127.0.0.1 - - [01/Oct/2018 08:52:49] "GET /api/v1/tileset_info/?d=a HTTP/1.1" 200 -


ret: 200 b'{\n  "a": {\n    "bins_per_dimension": 256, \n    "max_pos": [\n      2.5, \n      2.5\n    ], \n    "max_width": 5, \n    "max_zoom": 50, \n    "min_pos": [\n      -2.5, \n      -2.5\n    ], \n    "mirror_tiles": "false"\n  }\n}\n'
returning
CPU times: user 26.1 ms, sys: 44.4 ms, total: 70.5 ms
Wall time: 398 ms


127.0.0.1 - - [01/Oct/2018 08:52:49] "GET /api/v1/tiles/?d=a.0.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2018 08:52:54] "OPTIONS /api/v1/tileset_info/?d=a&s=b74nMmoOR0--llvmYifo3g HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2018 08:52:54] "OPTIONS /api/v1/tileset_info/?d=b&s=b74nMmoOR0--llvmYifo3g HTTP/1.1" 200 -


info: {'a': {'min_pos': [-2.5, -2.5], 'max_pos': [2.5, 2.5], 'bins_per_dimension': 256, 'max_width': 5, 'max_zoom': 50, 'mirror_tiles': 'false'}}


127.0.0.1 - - [01/Oct/2018 08:52:54] "GET /api/v1/tileset_info/?d=a&s=b74nMmoOR0--llvmYifo3g HTTP/1.1" 200 -
127.0.0.1 - - [01/Oct/2018 08:52:55] "GET /api/v1/tileset_info/?d=b&s=b74nMmoOR0--llvmYifo3g HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/Users/pete/miniconda3/envs/cenv3/lib/python3.6/site-packages/flask/app.py", line 1997, in __call__
    return self.wsgi_app(environ, start_response)
  File "/Users/pete/miniconda3/envs/cenv3/lib/python3.6/site-packages/flask/app.py", line 1985, in wsgi_app
    response = self.handle_exception(e)
  File "/Users/pete/miniconda3/envs/cenv3/lib/python3.6/site-packages/flask_cors/extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/pete/miniconda3/envs/cenv3/lib/python3.6/site-packages/flask/app.py", line 1540, in handle_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/pete/miniconda3/envs/cenv3/lib/python3.6/site-packages/flask/_compat.py", line

In [24]:
#%time tilesets[0]['handlers']['tiles'](['a.1.0.0', 'a.1.0.1'])
server.tiles('a', 0, 0, 0)

{'dense': '...',
 'dtype': 'float16'}

In [25]:
server.process

<Process(Process-5, started)>

In [26]:
import json

conf = hgc.HiGlassConfig()
view = conf.add_view(x=0,y=0, width=6, height=12)
view.add_track('a', 'heatmap', 'center', 
               server=server.api_address,
               height=400, options={
               })
view.add_track(None, 'top-axis', 'top')
view.add_track(None, 'left-axis', 'left')

view2 = conf.add_view(x=6,y=0, width=6, height=12)
view2.add_track('b', 'heatmap', 'center', 
               server=server.api_address)

#print(hgc.to_json_string())
import higlass_jupyter
higlass_jupyter.HiGlassDisplay(viewconf=conf.to_json_string(), 
                               hg_options='{"bounded": false, "renderer": "canvas"}')

A Jupyter Widget